In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import numpy as np

from bayes_opt import BayesianOptimization
from isic_helper import compute_pauc, compute_auc

In [2]:
def blend_optimizer(oof_preds_df, oof_columns, folds, init_points = 20, n_iter = 100):
    pbounds = {f"w{i}": (0.0, 10.0) for i in range(len(oof_columns))}

    def dim_opt(oof_preds_df, *args):
        weights = args
        score = 0
        for fold in folds:
            fold_ensemble_preds = 0
            for weight, oof_column in zip(weights, oof_columns):
                fold_ensemble_preds += weight * oof_preds_df.loc[oof_preds_df["fold"] == fold, oof_column].rank(pct=True).values
            score += compute_pauc(np.array(oof_preds_df.loc[oof_preds_df["fold"] == fold, "target"]), fold_ensemble_preds)
        return score / len(folds)

    def q(**ws):
        ws = tuple(ws.values())
        return dim_opt(oof_preds_df, *ws)

    optimizer = BayesianOptimization(
        f=q,
        pbounds=pbounds,
        random_state=2022,
    )

    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )
    
    weights = []
    for i in range(len(oof_columns)):
        weights.append(optimizer.max["params"][f"w{i}"])
    
    print(f"Best pAUC: {optimizer.max['target']}")
    print(f"Best weights: {weights}")
    return weights

In [3]:
id_column = "isic_id"
target_column = "target"
ensemble_column = "oof_ensemble"
fold_column = "gkf_fold"

boosting_model_names = ["xgb", "xgb", "xgb", "lgb", "cb"]
boosting_versions = ["v1", "v4", "v5", "v6", "v1"]
boosting_modes = ["train", "train", "train", "train", "train"]

cnn_model_names = ["efficientnet_b0", "efficientnet_b1"]
cnn_versions = ["v2", "v1"]
cnn_modes = ["trainbinary", "trainbinary"]

model_names = boosting_model_names + cnn_model_names
versions = boosting_versions + cnn_versions
modes = boosting_modes + cnn_modes
paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-{mode}" for model_name, version, mode in zip(model_names, versions, modes)]

val_auc_scores = {}
val_pauc_scores = {}
oof_columns = []
for idx, path in enumerate(paths):
    model_name = model_names[idx]
    version = versions[idx]
    mode = modes[idx]
    oof_preds_model_df = pd.read_csv(f"{path}/oof_train_preds_{model_name}_{version}.csv")
    if mode in ["trainbinary", "trainmulti"]:
        oof_column = f"oof_{model_name}_{version}_{mode}"
        oof_preds_model_df = oof_preds_model_df.rename(columns={f"oof_{model_name}_{version}": oof_column})
    else:
        oof_column = f"oof_{model_name}_{version}"
    if idx == 0:
        oof_preds_df = oof_preds_model_df.copy()
    else:
        oof_preds_df = oof_preds_df.merge(oof_preds_model_df[[id_column, oof_column]], on="isic_id", how="inner")
        assert oof_preds_df.shape[0] == oof_preds_model_df.shape[0]
    
    val_auc_scores[f"{model_name}_{version}_{mode}"] = {}
    val_pauc_scores[f"{model_name}_{version}_{mode}"] = {}
    oof_columns.append(oof_column)

all_folds = np.unique(oof_preds_df["fold"])
weights = blend_optimizer(
    oof_preds_df, oof_columns, all_folds,
    init_points=50, n_iter=100
)

|   iter    |  target   |    w0     |    w1     |    w2     |    w3     |    w4     |    w5     |    w6     |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.184     | 0.09359   | 4.991     | 1.134     | 0.4997    | 6.854     | 4.87      | 8.977     |
| 2         | 0.1849    | 6.475     | 8.97      | 7.211     | 8.314     | 8.276     | 8.336     | 9.57      |
| 3         | 0.1846    | 3.68      | 4.948     | 3.395     | 6.194     | 9.775     | 0.9643    | 7.442     |
| 4         | 0.1847    | 2.925     | 2.987     | 7.525     | 0.1866    | 5.237     | 8.644     | 3.888     |
| 5         | 0.1845    | 2.122     | 4.752     | 5.647     | 3.494     | 9.759     | 0.3782    | 7.943     |
| 6         | 0.184     | 3.579     | 7.48      | 9.145     | 3.727     | 9.649     | 0.8139    | 0.4245    |
| 7         | 0.1848    | 2.968     | 3.637     | 4.903     | 6.685     | 6.734     | 5.721     | 0.8059    |
| 8       

In [4]:
pprint(oof_columns)

['oof_xgb_v1',
 'oof_xgb_v4',
 'oof_xgb_v5',
 'oof_lgb_v6',
 'oof_cb_v1',
 'oof_efficientnet_b0_v2_trainbinary',
 'oof_efficientnet_b1_v1_trainbinary']


In [5]:
pprint(weights)

[10.0,
 7.3361504442122945,
 8.571338655949184,
 4.338410245675795,
 9.090738072249648,
 9.548410067383287,
 5.449950432866181]


In [6]:
val_auc_scores["ensemble"] = {}
val_pauc_scores["ensemble"] = {}
for fold in all_folds:
    fold_index = oof_preds_df[oof_preds_df["fold"] == fold].index
    fold_target = oof_preds_df.loc[fold_index, target_column]
    fold_ensemble_preds = 0
    for model_name, version, mode, oof_column, weight in zip(model_names, versions, modes, oof_columns, weights):
        fold_model_preds = oof_preds_df.loc[fold_index, oof_column]
        fold_ensemble_preds += fold_model_preds.rank(pct=True).values * weight 
        
        val_auc_scores[f"{model_name}_{version}_{mode}"][f"fold_{fold}"] = compute_auc(fold_target, fold_model_preds)
        val_pauc_scores[f"{model_name}_{version}_{mode}"][f"fold_{fold}"] = compute_pauc(fold_target, fold_model_preds, min_tpr=0.8)
    
    oof_preds_df.loc[fold_index, ensemble_column] = fold_ensemble_preds
    val_auc_scores["ensemble"][f"fold_{fold}"] = compute_auc(fold_target, fold_ensemble_preds)
    val_pauc_scores["ensemble"][f"fold_{fold}"] = compute_pauc(fold_target, fold_ensemble_preds, min_tpr=0.8)

for model_name, version, mode, oof_column, weight in zip(model_names, versions, modes, oof_columns, weights):
    print(f"Model: {model_name}_{version}_{mode} | Weightage: {weight}")
    
    print("Val AUC scores:")
    pprint(val_auc_scores[f"{model_name}_{version}_{mode}"])
    print("Val PAUC scores:")
    pprint(val_pauc_scores[f"{model_name}_{version}_{mode}"])
    
    cv_model_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[oof_column])
    cv_model_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[oof_column], min_tpr=0.8)

    cv_model_auc_avg = np.mean(list(val_auc_scores[f"{model_name}_{version}_{mode}"].values()))
    cv_model_pauc_avg = np.mean(list(val_pauc_scores[f"{model_name}_{version}_{mode}"].values()))

    cv_model_auc_std = np.std(list(val_auc_scores[f"{model_name}_{version}_{mode}"].values()))
    cv_model_pauc_std = np.std(list(val_pauc_scores[f"{model_name}_{version}_{mode}"].values()))
    
    print(f"CV AUC OOF: {cv_model_auc_oof}")
    print(f"CV PAUC OOF: {cv_model_pauc_oof}")
    print(f"CV AUC AVG: {cv_model_auc_avg}")
    print(f"CV PAUC AVG: {cv_model_pauc_avg}")
    print(f"CV AUC STD: {cv_model_auc_std}")
    print(f"CV PAUC STD: {cv_model_pauc_std}")
    print("\n")

print("Val AUC scores:")
pprint(val_auc_scores["ensemble"])
print("Val PAUC scores:")
pprint(val_pauc_scores["ensemble"])

cv_ensemble_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[ensemble_column])
cv_ensemble_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[ensemble_column], min_tpr=0.8)

cv_ensemble_auc_avg = np.mean(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_avg = np.mean(list(val_pauc_scores["ensemble"].values()))

cv_ensemble_auc_std = np.std(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_std = np.std(list(val_pauc_scores["ensemble"].values()))

print(f"CV AUC OOF: {cv_ensemble_auc_oof}")
print(f"CV PAUC OOF: {cv_ensemble_pauc_oof}")
print(f"CV AUC AVG: {cv_ensemble_auc_avg}")
print(f"CV PAUC AVG: {cv_ensemble_pauc_avg}")
print(f"CV AUC STD: {cv_ensemble_auc_std}")
print(f"CV PAUC STD: {cv_ensemble_pauc_std}")
print(f"CV PAUC RANGE: ({cv_ensemble_pauc_avg - cv_ensemble_pauc_std}, {cv_ensemble_pauc_avg + cv_ensemble_pauc_std})")

Model: xgb_v1_train | Weightage: 10.0
Val AUC scores:
{'fold_1': 0.9878518102684217,
 'fold_2': 0.9746944700959227,
 'fold_3': 0.9596700247095382,
 'fold_4': 0.9790326153423768,
 'fold_5': 0.9804720485526146}
Val PAUC scores:
{'fold_1': 0.19063392900696513,
 'fold_2': 0.17969622036934285,
 'fold_3': 0.16585801687237148,
 'fold_4': 0.18230998302779194,
 'fold_5': 0.18376045977408706}
CV AUC OOF: 0.8940107119864256
CV PAUC OOF: 0.14372683159384408
CV AUC AVG: 0.9763441937937747
CV PAUC AVG: 0.1804517218101117
CV AUC STD: 0.009353518430939187
CV PAUC STD: 0.008144019048404384


Model: xgb_v4_train | Weightage: 7.3361504442122945
Val AUC scores:
{'fold_1': 0.9774012004442393,
 'fold_2': 0.9585113204613043,
 'fold_3': 0.9635616209706606,
 'fold_4': 0.9713384957132696,
 'fold_5': 0.9692148821814502}
Val PAUC scores:
{'fold_1': 0.18439364147339235,
 'fold_2': 0.16635088150424956,
 'fold_3': 0.17198002333678597,
 'fold_4': 0.174545119866219,
 'fold_5': 0.17352956799065142}
CV AUC OOF: 0.947856

In [7]:
oof_preds_df.pivot_table(index="target", values=oof_columns + [ensemble_column], aggfunc="mean")

,oof_cb_v1,oof_efficientnet_b0_v2_trainbinary,oof_efficientnet_b1_v1_trainbinary,oof_ensemble,oof_lgb_v6,oof_xgb_v1,oof_xgb_v4,oof_xgb_v5
target,,,,,,,,
0,0.009339,0.00229,0.000992,27.142693,0.010528,0.017897,0.014427,0.022984
1,0.423538,0.32989,0.230465,52.803446,0.455325,0.141498,0.329018,0.345672


In [8]:
oof_preds_df[oof_columns + [ensemble_column]].corr()

,oof_xgb_v1,oof_xgb_v4,oof_xgb_v5,oof_lgb_v6,oof_cb_v1,oof_efficientnet_b0_v2_trainbinary,oof_efficientnet_b1_v1_trainbinary,oof_ensemble
oof_xgb_v1,1.000000,0.138591,0.303888,0.273691,0.298925,0.211955,0.204397,0.067710
oof_xgb_v4,0.138591,1.000000,0.778952,0.711025,0.674799,0.485877,0.403045,0.331027
oof_xgb_v5,0.303888,0.778952,1.000000,0.655779,0.632031,0.429455,0.357309,0.332617
oof_lgb_v6,0.273691,0.711025,0.655779,1.000000,0.927002,0.573779,0.458106,0.363016
oof_cb_v1,0.298925,0.674799,0.632031,0.927002,1.000000,0.616194,0.495410,0.350483
oof_efficientnet_b0_v2_trainbinary,0.211955,0.485877,0.429455,0.573779,0.616194,1.000000,0.765765,0.151669
oof_efficientnet_b1_v1_trainbinary,0.204397,0.403045,0.357309,0.458106,0.495410,0.765765,1.000000,0.092857
oof_ensemble,0.067710,0.331027,0.332617,0.363016,0.350483,0.151669,0.092857,1.000000
